### The domain

First, we'll create our computational domain -- a regular mesh of the unit square with quadrilateral elements.

In [ ]:
import firedrake
from firedrake import Constant, inner, grad, dot, dx, ds
import numpy as np
import matplotlib.pyplot as plt

nx, nz = 32, 32
computational_domain = firedrake.UnitSquareMesh(nx, nz, quadrilateral=True)

Our physical domain will be obtained by warping the coordinates, like we saw in the thermochronometry notebook.
Here I've wrapped up some of the functions to create the bed topography because we'll have to do so more than once.

In [ ]:
δb = Constant(1 / 4)
α = Constant(1 / 8)
ξ_0 = Constant(1 / 2)

def sech(z):
    return 2 / (firedrake.exp(z) + firedrake.exp(-z))

def bed_topography(ξ):
    return δb * sech((ξ - ξ_0) / α)

In [ ]:
ξ = firedrake.SpatialCoordinate(computational_domain)
bed_expr = bed_topography(ξ[0])
surf_expr = Constant(1.0)
expr = firedrake.as_vector((ξ[0], (1 - ξ[1]) * bed_expr + ξ[1] * surf_expr))

Vc = computational_domain.coordinates.function_space()
X = firedrake.Function(Vc)
X.interpolate(expr)
domain = firedrake.Mesh(X, reorder=False)

### Initial solution

Let's try solving this using continuous Galerkin basis functions like we've done before.

In [ ]:
cg1 = firedrake.FiniteElement("CG", "quadrilateral", 1)
Q_cg = firedrake.FunctionSpace(domain, cg1)

In [ ]:
x = firedrake.SpatialCoordinate(domain)

expr = (x[0] * (1 - x[0]) * (1 - x[1]) * (x[1] - bed_topography(x[0]))) ** 2
Ψ = firedrake.Function(Q_cg).interpolate(expr)

In [ ]:
fig, ax = plt.subplots()
colors = firedrake.tripcolor(Ψ, axes=ax)
fig.colorbar(colors);

In [ ]:
grad_Ψ = firedrake.grad(expr)
V = firedrake.VectorFunctionSpace(domain, cg1)
u_expr = firedrake.as_vector((-grad_Ψ[1], grad_Ψ[0]))
u = firedrake.Function(V).interpolate(u_expr)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.quiver(u, axes=ax)
fig.colorbar(colors);

In [ ]:
import irksome
from irksome import Dt
import tqdm

In [ ]:
x_0 = firedrake.Constant((0.25, 0.55))
r = firedrake.Constant(0.25)
q_expr = firedrake.exp(-inner(x - x_0, x - x_0) / r**2)

q = firedrake.Function(Q_cg)
q.interpolate(q_expr);

In [ ]:
max_speed = np.abs(u.dat.data_ro).flatten().max()
domain_size = 1.0
final_time = 2 * domain_size / max_speed
min_cell_size = domain.cell_sizes.dat.data_ro.min()
timestep = min_cell_size / max_speed / 8
num_steps = int(final_time / timestep)

print(f"Final time: {final_time:.2f}")
print(f"Timestep:   {timestep:.2f}")

Once again, I'm wrapping up the important stuff in a function that I can reuse.
The `degree=4` bit at the end of the form is to use a lower-order quadrature rule; without this, Firedrake will try to estimate the quadrature degree and will pick something ridiculous.

In [ ]:
t = Constant(0.0)
dt = Constant(timestep)

ϕ = firedrake.TestFunction(Q_cg)
F = (Dt(q) * ϕ - q * inner(u, grad(ϕ))) * dx
method = irksome.BackwardEuler()
solver = irksome.TimeStepper(F, method, t, dt, q)

qs_cg = [q.copy(deepcopy=True)]
for step in tqdm.trange(num_steps):
    solver.advance()
    qs_cg.append(q.copy(deepcopy=True))

That does not look good.
It doesn't get much better with finer resolution either.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(qs_cg[-1], axes=ax)
fig.colorbar(colors);

### Discontinuous Galerkin

Now let's try another basis.
The variational form becomes
$$\int_\Omega\left(\partial_tq\cdot\phi - qu\cdot\nabla\phi\right)dx + \sum_\Gamma\int_\Gamma(f_+\cdot - f_-)(\phi_+ - \phi_-)d\gamma = 0$$

In [ ]:
from firedrake import dS

dg = firedrake.FiniteElement("DQ", "quadrilateral", 0)
Q_dg = firedrake.FunctionSpace(domain, dg)

q = firedrake.Function(Q_dg).project(q_expr)

ϕ = firedrake.TestFunction(Q_dg)

ν = firedrake.FacetNormal(domain)
u_ν = firedrake.max_value(0, inner(u, ν))
f = q * u_ν

F = (Dt(q) * ϕ - q * inner(u, grad(ϕ))) * dx + (f("+") - f("-")) * (ϕ("+") - ϕ("-")) * dS
method = irksome.BackwardEuler()
solver = irksome.TimeStepper(F, method, t, dt, q)

qs_dg = [q.copy(deepcopy=True)]
for step in tqdm.trange(num_steps):
    solver.advance()
    qs_dg.append(q.copy(deepcopy=True))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(qs_dg[-1], axes=ax)
fig.colorbar(colors);

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(qs_dg[0], vmin=0, vmax=1, num_sample_points=1, axes=ax);
fn_plotter = firedrake.FunctionPlotter(domain, num_sample_points=1)

def animate(q):
    colors.set_array(fn_plotter(q))

animation = FuncAnimation(fig, animate, qs_dg, interval=1e3/30)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())